In [1]:
import pandas as pd
import folium
import seaborn as sns
import json

In [2]:
import matplotlib.pyplot as plt
plt.rc('axes', unicode_minus=False)
plt.rc('font', family='Malgun Gothic')
%matplotlib inline

In [3]:
from IPython.display import set_matplotlib_formats
# 폰트가 선명하게 보이기 위해
set_matplotlib_formats('retina')

In [4]:
df_hospital = pd.read_csv("./gangnam_hospital.csv", low_memory=False)

In [5]:
dongDf = pd.read_csv('./data/강남구노인인구.csv', index_col='동')  
dongDf.columns = dongDf.columns.map(str); dongDf
dongDevided = json.load(open( './data/gangnam_map.json', encoding = 'utf-8'))

In [6]:
dongMap = folium.Map(location=[df_hospital["위도"].mean(), df_hospital["경도"].mean()], zoom_start = 12.5, 
                   tiles = 'openstreetmap'); dongMap
fmap = folium.Choropleth(geo_data = dongDevided,
                         data = dongDf,
                         columns = ['행정구역', '계'],
                         key_on = 'feature.properties.adm_nm', fill_color='YlOrRd',
                         fill_opacity = 0.7, line_opacity = 0.3,
                         legend_name = '노인 인구수',
                         highlight=True
                         
                        ).add_to(dongMap)

folium.LayerControl().add_to(dongMap)

# 병원 
loc = df_hospital[["위도", "경도"]].values[:len(df_hospital)].tolist()
for i in range(len(df_hospital)):
    folium.Circle(location=loc[i], radius=5, color="green", tooltip = df_hospital['상호명'][i], fill = True).add_to(dongMap)


fmap.geojson.zoom_on_click = False
fmap.geojson.add_child(
     folium.features.GeoJsonTooltip(['adm_nm'],labels = False)
)
dongMap.save('./dongMap.html'); dongMap